In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
import csv

In [2]:
dataDir='annotations'
dataType='val'
annFile='{}/COCO2017/instances_{}2017.json'.format(dataDir,dataType)
coco = COCO(annFile)

loading annotations into memory...
Done (t=0.79s)
creating index...
index created!


In [3]:
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush



In [4]:
catIdsCOCO = []
class_counts = {}
annotations = []
total_images = 0

def add_cat_to_annotations(catIds, class_name):
    count = 0 # count of annotations in category
    img_count = 0 # count of images
    imgIds = coco.getImgIds(catIds=catIds[-1])
    for imgId in imgIds:
        img_count += 1
        img = coco.loadImgs(imgId)
        annIds = coco.getAnnIds(imgIds=img[0]['id'], catIds=catIds[-1], iscrowd=None)
        anns = coco.loadAnns(annIds)
        count += len(anns)
        for a in anns:
            # image filename, width, height, class, xmin, xmax, ymin, ymax
            info = {'file_name':img[0]['file_name'], 'width':img[0]['width'], 'height':img[0]['height'], 'class':c, 'xmin':a['bbox'][0], 'xmax':a['bbox'][0]+a['bbox'][2], 'ymin':a['bbox'][1], 'ymax':a['bbox'][1]+a['bbox'][3]}
            annotations.append(info)
            #if class_name not in class_anns:
            #    class_anns[class_name] = [info]
            #else:
            #    class_anns[class_name].append(info)
    return count, img_count

for c in nms:
    catIds = coco.getCatIds(catNms=c)
    count, img_count = add_cat_to_annotations(catIds, c)
    total_images += img_count
    catIdsCOCO.append(catIds)
    if c in class_counts:
        class_counts[c] += count
    else:
        class_counts[c] = count

print('Number of annotations: %s' % len(annotations))
print('Number of classes: %s' % len(class_counts))
print('Class counts: %s' % class_counts)
print('Number of classes: %s' % len(catIdsCOCO))
print('Total number of images: %s' % total_images)

Number of annotations: 36781
Number of classes: 80
Class counts: {'person': 11004, 'bicycle': 316, 'car': 1932, 'motorcycle': 371, 'airplane': 143, 'bus': 285, 'train': 190, 'truck': 415, 'boat': 430, 'traffic light': 637, 'fire hydrant': 101, 'stop sign': 75, 'parking meter': 60, 'bench': 413, 'bird': 440, 'cat': 202, 'dog': 218, 'horse': 273, 'sheep': 361, 'cow': 380, 'elephant': 255, 'bear': 71, 'zebra': 268, 'giraffe': 232, 'backpack': 371, 'umbrella': 413, 'handbag': 540, 'tie': 254, 'suitcase': 303, 'frisbee': 115, 'skis': 241, 'snowboard': 69, 'sports ball': 263, 'kite': 336, 'baseball bat': 146, 'baseball glove': 148, 'skateboard': 179, 'surfboard': 269, 'tennis racket': 225, 'bottle': 1025, 'wine glass': 343, 'cup': 899, 'fork': 215, 'knife': 326, 'spoon': 253, 'bowl': 626, 'banana': 379, 'apple': 239, 'sandwich': 177, 'orange': 287, 'broccoli': 316, 'carrot': 371, 'hot dog': 127, 'pizza': 285, 'donut': 338, 'cake': 316, 'chair': 1791, 'couch': 261, 'potted plant': 343, 'bed':

In [5]:
# write to csv
with open('test_fullcoco.csv', mode='w') as csv_file:
    fieldnames = ['file_name', 'width', 'height', 'class', 'xmin', 'xmax', 'ymin', 'ymax']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for ann in annotations:
        writer.writerow(ann)